In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import os
from torchvision.io import read_image
from PIL import Image
import pandas as pd

# importing the libraries
import pandas as pd
import numpy as np

# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout, Conv3d
from torch.optim import Adam, SGD


niftypath = "/mnt/c/Users/Patrick/Documents/MPHYS_DATA_NIFTY"
croppath = "/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED"

Error: Session cannot generate requests

In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
print(test_data)

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

In [2]:
idx = 0
transform = ToTensor()

class CustomImageDataset():
    def __init__(self, croppath, transform=None, target_transfrom=None):
        #self.img_labels = pd.read_csv("/mnt/c/Users/Patrick/Documents/NSCLC Radiomics Lung1.clinical-version3-Oct 2019(1).csv")
        self.img_labels = pd.read_csv("/mnt/c/Users/Patrick/Documents/cancerdata.csv")
        #print(self.img_labels)
        self.img_dir = croppath
        self.transform=transform
        #print(self.transform)
        self.target_transform=target_transfrom

    def __len__ (self):
        return len(self.img_labels)

    def __getitem__ (self, idx):
        img_path = os.path.join(self.img_dir, f"{self.img_labels.iloc[idx, 0]}-RTSTRUCT.nii")
        print(img_path)
        image = sitk.ReadImage(img_path)
        array = sitk.GetArrayFromImage(image)
        tensor = torch.from_numpy(array)
        time_to_death = self.img_labels.iloc[idx,8]
        dead_status = self.img_labels.iloc[idx,9]
        if time_to_death < 1.5*365 and dead_status == 1:
            label=1
        else:
            label=0
        #print(label)
        return(tensor, label)
dataset = CustomImageDataset(croppath, ToTensor(), None)
#print(len(dataset))

trainset, valset, testset = torch.utils.data.random_split(dataset, [14,3,3])
# for i in range(14):
#     train_x = trainset.__getitem__(i)[0]
#     train_y = trainset.__getitem__(i)[1]

# for i in range(3):
#     val_x = trainset.__getitem__(i)[0]
#     val_y = trainset.__getitem__(i)[1]
#     test_x = trainset.__getitem__(i)[0]
#     test_y = trainset.__getitem__(i)[1]




In [3]:
xyz = dataset.__getitem__(0)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
z,x,y = np.argwhere(xyz)
ax.scatter(x,y,z,c=z,alpha = 1)
plt.show()

In [3]:
train_dataloader = DataLoader(trainset, batch_size=4, shuffle=True)
train_features = next(iter(train_dataloader))
test_dataloader = DataLoader(testset, batch_size=4, shuffle=True)
test_features = next(iter(train_dataloader))
val_dataloader = DataLoader(valset, batch_size=4, shuffle=True)
val_features = next(iter(val_dataloader))


/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-018-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-014-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-005-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-017-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-018-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-003-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-011-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-015-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-002-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-016-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-001-RTSTRUCT.nii


In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv3d(178*178*178, 8, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),#normalises batch
            ReLU(inplace=True),#applies a ReLu to the neurons
            MaxPool2d(kernel_size=2, stride=2),#finds max pool of feature map
            # Defining another 2D convolution layer
            Conv3d(8, 8, kernel_size=3, stride=1, padding=1),#64 neurons per layer
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(4 * 7 * 7, 10)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)#calls the constructor to execute the convolutions, passes the tensor x and gets the 
        #result of the convolution passed back.
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x




cpu


In [23]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

In [ ]:
def train(epoch):
    model.train()
    tr_loss = 0
    x_train = []
    y_train = []
    # getting the training set
    for i in range(14):
        x_train.append(trainset.__getitem__(i)[0])
        y_train.append(trainset.__getitem__(i)[1])
    # getting the validation set
    for i in range(3):
        x_val = valset.__getitem__(i)[0]
        y_val = valset.__getitem__(i)[1]
    
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = model(x_train)
    output_val = model(x_val)

    # computing the training and validation loss
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

Error: Session cannot generate requests

In [17]:
# defining the number of epochs
n_epochs = 25
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# training the model
for epoch in range(n_epochs):
    train(epoch)

/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-019-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-019-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-011-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-011-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-010-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-010-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-020-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-020-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-003-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-003-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-012-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-012-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/LUNG1-008-RTSTRUCT.nii
/mnt/c/Users/Patrick/Documents/MPHYS_DATA_CROPPED/L

TypeError: conv2d() received an invalid combination of arguments - got (list, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!list!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!list!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)


In [ ]:
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()
      self.conv1 = nn.Conv2d(1, 32, 3, 1)
      self.conv2 = nn.Conv2d(32, 64, 3, 1)
      self.dropout1 = nn.Dropout2d(0.25)
      self.dropout2 = nn.Dropout2d(0.5)
      self.fc1 = nn.Linear(9216, 128)
      self.fc2 = nn.Linear(128, 10)

    # x represents our data
    def forward(self, x):
      # Pass data through conv1
      x = self.conv1(x)
      # Use the rectified-linear activation function over x
      x = F.relu(x)

      x = self.conv2(x)
      x = F.relu(x)

      # Run max pooling over x
      x = F.max_pool2d(x, 2)
      # Pass data through dropout1
      x = self.dropout1(x)
      # Flatten x with start_dim=1
      x = torch.flatten(x, 1)
      # Pass data through fc1
      x = self.fc1(x)
      x = F.relu(x)
      x = self.dropout2(x)
      x = self.fc2(x)

      # Apply softmax to x
      output = F.log_softmax(x, dim=1)
      return output